In [0]:
"""
In this notebook , i implemented a small similar project i did before.
We had a project for language(Say German) with NLP and 
we were asked to convert it into another language(Chinese)
Since we do not know second language and for some reasons,
1st we convert corpus with google translate.
Then from verb-noun pairs we tried to understand different meanings for verbs.

text1 = "He played very good football" 
text2 = "He played tennis while he was in Tokyo" 
text3 = "He played violin in high school" 
text4 = "He played king in theatre" 

for above sentences verb is play but meanings are different.
We tried to do this task with word2vec and ELmo.
Since Elmo generates embedings with context, it was much better.

We found verb-noun patterns, and check similarity of nouns, to understand
the number of  different meanings of a given verb. 

"""
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
import tensorflow as tf

pd.set_option('display.max_colwidth', 200)
tf.compat.v1.enable_eager_execution()


In [0]:
#!pip install textacy

In [0]:
nlp = spacy.load("en_core_web_sm")

In [28]:
text1 = "He played very good football" 
text2 = "He played tennis while he was in Tokyo" 
text3 = "He played violin in high school" 
text4 = "He played king in theatre" 
texts = [text1,text2,text3,text4]
def describe(sentence):
  print( "\nsentence :",sentence)
  doc = nlp(sentence)
  for tok in doc:
    print(tok.text, "-->",tok.dep_,"-->", tok.pos_)

for t in texts:
  describe(t)  


sentence : He played very good football
He --> nsubj --> PRON
played --> ROOT --> VERB
very --> advmod --> ADV
good --> amod --> ADJ
football --> dobj --> NOUN

sentence : He played tennis while he was in Tokyo
He --> nsubj --> PRON
played --> ROOT --> VERB
tennis --> dobj --> NOUN
while --> mark --> SCONJ
he --> nsubj --> PRON
was --> advcl --> AUX
in --> prep --> ADP
Tokyo --> pobj --> PROPN

sentence : He played violin in high school
He --> nsubj --> PRON
played --> ROOT --> VERB
violin --> dobj --> NOUN
in --> prep --> ADP
high --> amod --> ADJ
school --> pobj --> NOUN

sentence : He played king in theatre
He --> nsubj --> PRON
played --> ROOT --> VERB
king --> dobj --> NOUN
in --> prep --> ADP
theatre --> pobj --> PROPN


In [7]:
import spacy,en_core_web_sm
import textacy
nlp = en_core_web_sm.load()

#Catch verb-noun pairs
pattern = [{'LOWER': 'played'},
           #{'POS': 'ADV', 'OP': '*'},
           {'OP': '*'},
           {'POS': 'NOUN', 'OP': '+'}]

for sentence in texts:
  doc = textacy.make_spacy_doc(sentence, lang='en_core_web_sm')
  lists = textacy.extract.matches(doc, pattern)
  for list in lists:
    print(list.text)

played very good football
played tennis
played violin
played violin in high school
played king


In [8]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

#get a word2vec corpus
corpus = api.load('text8')  
model = Word2Vec(corpus)  
model.most_similar("car")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('driver', 0.8096974492073059),
 ('truck', 0.7256351113319397),
 ('motorcycle', 0.7204147577285767),
 ('taxi', 0.7120797634124756),
 ('cars', 0.707251787185669),
 ('vehicle', 0.6724898815155029),
 ('passenger', 0.6630937457084656),
 ('racing', 0.6602973937988281),
 ('automobile', 0.6409180760383606),
 ('glider', 0.634545087814331)]

In [9]:
#we can see that tennis,football is much related than others
keywords = ["tennis","football","violin","heart","theater","card","king"]
for i,key1 in enumerate(keywords):
  for j,key2 in enumerate(keywords):
    if i < j :
      print( key1 ," <> " , key2 , model.similarity(key1,key2))

tennis  <>  football 0.7245322
tennis  <>  violin 0.35113767
tennis  <>  heart 0.07257649
tennis  <>  theater 0.26850238
tennis  <>  card 0.34463185
tennis  <>  king -0.046462316
football  <>  violin 0.22081622
football  <>  heart -0.053363036
football  <>  theater 0.20171845
football  <>  card 0.4159222
football  <>  king 0.008354831
violin  <>  heart -0.0027932727
violin  <>  theater 0.20187119
violin  <>  card 0.2113994
violin  <>  king -0.02630376
heart  <>  theater 0.14609508
heart  <>  card 0.060182434
heart  <>  king 0.09100635
theater  <>  card 0.06485075
theater  <>  king -0.004532304
card  <>  king -0.002638336


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [10]:
import tensorflow_hub as hub
#download elmo
elmo_module_url = "https://tfhub.dev/google/elmo/2"
elmo_embed = hub.load(elmo_module_url)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

"""
    The return values of elmo
    word_emb: the character-based word representations with shape [batch_size, max_length, 512].
    lstm_outputs1: the first LSTM hidden state with shape [batch_size, max_length, 1024].
    lstm_outputs2: the second LSTM hidden state with shape [batch_size, max_length, 1024].
    elmo: the weighted sum of the 3 layers, where the weights are trainable. This tensor has shape [batch_size, max_length, 1024]
    default: a fixed mean-pooling of all contextualized word representations with shape [batch_size, 1024].

"""

In [43]:
#get a sentence embedding
def elmo_vectors(x):
  q = tf.constant([x])
  embeddings = elmo_embed.signatures['default'](q)
  return embeddings["default"].numpy()

#get an embedding of word
def elmo_vectors_word(x,index):
  q = tf.constant([x])
  embeddings = elmo_embed.signatures['default'](q)
  return embeddings["elmo"].numpy()[0][index].reshape(1,1024) 


v1 = elmo_vectors_word("he played football",2)
v2 = elmo_vectors_word("he played tennis",2)
v3 = elmo_vectors_word("he played violin",2)

print("sentences of football,tennis ", cosine_similarity(v1,v2))
print("sentences of football,violin ", cosine_similarity(v1,v3))
print("sentences of tennis,violin ", cosine_similarity(v2,v3))

sentences of football,tennis  [[0.79441893]]
sentences of football,violin  [[0.5626879]]
sentences of tennis,violin  [[0.5979855]]


In [47]:

elmo_embeddings=[]
print (len(texts))
for i in range(len(texts)):
    print (texts[i])
    elmo_embeddings.append(elmo_vectors(texts[i]).squeeze())

4
He played very good football
He played tennis while he was in Tokyo
He played violin in high school
He played king in theatre


In [55]:
relation_sentences = cosine_similarity(elmo_embeddings, elmo_embeddings)
print(relation_sentences)

[[1.         0.59570265 0.6094363  0.6669673 ]
 [0.59570265 0.99999994 0.62427634 0.6425481 ]
 [0.6094363  0.62427634 0.9999998  0.7093555 ]
 [0.6669673  0.6425481  0.7093555  1.        ]]


In [56]:
import pandas as pd
cols = ["football","tennis","violin","theatre"]
pd.DataFrame(relation_sentences,columns=cols,index=cols)

,football,tennis,violin,theatre
football,1.000000,0.595703,0.609436,0.666967
tennis,0.595703,1.000000,0.624276,0.642548
violin,0.609436,0.624276,1.000000,0.709355
theatre,0.666967,0.642548,0.709355,1.000000


In [61]:
elmo_embeddings_word=[]
print (len(texts))
for i in range(len(texts)):
    print (texts[i])
    elmo_embeddings_word.append(elmo_vectors_word(texts[i],2).squeeze())

4
He played very good football
He played tennis while he was in Tokyo
He played violin in high school
He played king in theatre


In [62]:
relation_words = cosine_similarity(elmo_embeddings_word, elmo_embeddings_word)
print(relation_words)

[[0.9999999  0.2986014  0.26330966 0.27360702]
 [0.2986014  1.0000001  0.5190791  0.32667577]
 [0.26330966 0.5190791  1.         0.34032404]
 [0.27360702 0.32667577 0.34032404 1.0000002 ]]


In [63]:
import pandas as pd
cols = ["football","tennis","violin","theatre"]
pd.DataFrame(relation_words,columns=cols,index=cols)

,football,tennis,violin,theatre
football,1.000000,0.298601,0.263310,0.273607
tennis,0.298601,1.000000,0.519079,0.326676
violin,0.263310,0.519079,1.000000,0.340324
theatre,0.273607,0.326676,0.340324,1.000000
